<a href="https://colab.research.google.com/github/perfectpanda-works/machine-learning/blob/master/LEARNING_PYTORCH_WITH_EXAMPLES3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Autograd

In [ ]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
#device = torch.device("cpu")
device = torch.device("cuda:0") # Uncomment this to run on GPU

print(device)

cuda:0


In [ ]:
# N  　：バッチサイズ
# D_in ：入力次元数
# H　　：隠れ層の次元数
# D_out：出力次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# ランダムな入力データと出力データの作成
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 重みをランダムに初期化
# 勾配を計算したいテンソルなので、「requires_grad=True」を指定しておく。
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

In [ ]:
#学習率
learning_rate = 1e-6

for t in range(500):
    # ①順伝播: 1行で表記
    # 手動の逆伝播で計算途中の結果を保持する必要がなくなった。
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # 損失
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # ②「autograd」機能を利用して逆伝播を行い勾配を計算します。
    #  backward実行後に「w1.grad」と「w2.grad」に勾配が入ります。
    loss.backward()

    # ③重みの更新時の計算は、「autograd」で追跡する必要がないので
    # 「autograd」を停止するため、「torch.no_grad()」で処理を囲みます。
    # 「torch.optim.SGD」を利用しても同じことができます。
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 手動で「grad」を初期化します。
        w1.grad.zero_()
        w2.grad.zero_()

99 368.0965270996094
199 1.209576964378357
299 0.005984519608318806
399 0.00016405261703766882
499 3.48710673279129e-05


#autogradをカスタマイズ

In [ ]:
# -*- coding: utf-8 -*-
import torch

class MyReLU(torch.autograd.Function):
    """
    サブクラス化することで、独自のカスタムautograd関数を実装できます。
    torch.autograd.Functionを継承
    """

    @staticmethod
    def forward(ctx, input):
        """
        順伝播では、「ctx」と「input」を受け取ります。
        「ctx」は逆伝播のために情報を保持しておくコンテキストオブジェクトです。
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        逆伝播では、出力側からの損失の勾配を含むTensorを受け取ります
        そして私たちは入力側からの損失の勾配を計算する必要があります
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("cuda:0") # Uncomment this to run on GPU
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    #定義したautogradを利用するために、「apply」を行います。
    relu = MyReLU.apply

    # 順伝播：定義したforwardの処理を利用
    y_pred = relu(x.mm(w1)).mm(w2)

    # 損失
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # 逆伝播
    loss.backward()

    # 重みの更新：勾配降下法
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

99 337.72589111328125
199 1.0916001796722412
299 0.006575985811650753
399 0.00017215670959558338
499 3.354191721882671e-05
